In [17]:
#We can go into our root file and see what Trees are availiable
%matplotlib inline
import sys, os
if __package__ is None:
    import sys, os
    sys.path.append(os.path.realpath("/data/shared/Software/"))
    sys.path.append(os.path.realpath("../../"))
import numpy as np
import pandas as pd
import ntpath
import glob
import deepconfig

from CMS_Deep_Learning.utils.metrics import plot_history, print_accuracy_m
from CMS_Deep_Learning.utils.callbacks import OverfitStopping, SmartCheckpoint

from keras.models import Sequential
from keras.layers import Dense, Flatten, Reshape, Activation, Dropout, Convolution2D
from keras.callbacks import EarlyStopping


dc = deepconfig.deepconfig(gpu='gpu1', backend='theano')

#The observables taken from the table
observ_types = ['E/c', 'Px', 'Py', 'Pz', 'PID', 'Charge', "PT", "Eta", "Phi", "Dxy", "Eem", "Ehad"]
set_size = 5
vecsize = len(observ_types)
epochs = 100
batch_size = 100

#http://pdg.lbl.gov/2012/reviews/rpp2012-rev-monte-carlo-numbering.pdf
observable_vocab = [-11,11,-13,13,22]
quark_vocab = [5,-5,6,-6]
neutrino_vocab = [12,-12, 14, -14, 16, -16, 18, -18]
unobservable_boson_vocab = [24, -24]
jet_vocab = [100, -100]
missing_ET_vocab = [83,84] #83 Reg, #84 Puppi 
Eflow_vocab = [89,90,91] #89 Track, #90 Photon #91 Hadron


#particle_vocab = observable_vocab + neutrino_vocab + quark_vocab +  unobservable_boson_vocab + jet_vocab + missing_ET_vocab + Eflow_vocab
particle_vocab = observable_vocab + missing_ET_vocab + Eflow_vocab
particle_dict = {particle_vocab[i]:i for i in range(len(particle_vocab))}
#print(particle_dict)
def cullNonObservables(frame):
    #Status of 1 means that the particle is a stable product
    stable_cond = frame["Status"] == 1 
    #All even leptons are neutrinos which we can't measure
    notNeutrino_cond = ((np.abs(frame["PID"]) != 12) & (np.abs(frame["PID"]) != 14) & (np.abs(frame["PID"]) != 16)) 
    #Get all entries that satisfy the conditions
    frame = frame[stable_cond & notNeutrino_cond]
    #Drop the Status frame since we only needed it to see if the particle was stable
    frame = frame.drop(["Status"], axis=1)
    return frame

def mapPIDS(observables):
    PIDS = observables["PID"] 
    observables["PID"] = PIDS.apply(lambda x: particle_dict[x])

def padItem(x, shuffle=False):
    if(len(x) > set_size):
        return x[:set_size]
    else:
        out = np.append(x ,np.array(np.zeros((set_size - len(x), vecsize))), axis=0)
        if(shuffle): np.random.shuffle(out)
        return out
def padInput(l,shuffle=False):
    #out = []
    for i,x in enumerate(l):
        #out.append(padItem(x, shuffle=shuffle))
        l[i] = padItem(x, shuffle=shuffle)
    return l

def helper_gPID(x, pidDict):
    pid = x.iloc[0]['PID']
    x = x.drop(["PID"], axis=1)
    pidDict[pid] = np.array(x)

def helper_gEntry(x, arr, groupByPID=True):
    if(groupByPID == True):
        pidDict = {}
        group = x.groupby(["PID"]).apply(lambda x: helper_gPID(x, pidDict))
        arr.append(np.array(pidDict))
    else:
        arr.append(np.array(x))
    return 0
def groupEntriesToArrays(frame, select, groupByPID=True):
    arr = []
    
    if(groupByPID == True):
        arr = {}
        grouped = frame.groupby(["Entry","PID"])[select]#.apply(lambda x: helper_gEntry(x,arr,groupByPID=groupByPID))
        keys = sorted(grouped.groups, key = lambda key:key[0])# [key for key in  grouped.groups].sort()
        for key in  keys:
            if((key[1] in arr) == False):
                arr[key[1]] = []
            arr[key[1]].append(grouped.get_group(key).values)
            #print(key)
            #print(grouped.get_group(key).values)
            #print(np.array(grouped.get_group(key)))
            #print()
        #print([(x, len(groups[x])) for x in groups])
    else:
        grouped = frame.groupby(["Entry"])[select].apply(lambda x: helper_gEntry(x,arr,groupByPID=groupByPID))
    #print(arr)
    return arr
def preprocessFromPandas_file_label_pairs(files, cull=False, groupByPID=True):
    X_train = {}
    y_train = []
    X_train_by_label = {}
    y_train_by_label = {}
    max_size = 0
    for f,label in files:
        all_particles = pd.read_hdf(f, 'data')
        cond = (all_particles["Entry"] < 2)
        all_particles = all_particles[cond]
        if(cull==True):
            observables = cullNonObservables(all_particles)
        else:
            observables = all_particles
        #print(observables)
        #cond = ((np.abs(observables["PID"]) == 11) | (np.abs(observables["PID"]) == 13) \
        #    | (np.abs(observables["PID"]) == 22) | (np.abs(observables["PID"]) == 83))
        #cond = (np.abs(observables["PID"]) != 91)&(observables["E/c"] > 10)
        #observables = observables[cond].sort(["PID"])
        #print(observables)
        #print(label, f)
        #print(frame_cond)
        
        #mapPIDS(observables)
        
       
            
        grouped_arrays = groupEntriesToArrays(observables, observ_types, groupByPID=groupByPID)
        #m = len(max(grouped_arrays,key=len))
       
        if(groupByPID == True):
            for key in grouped_arrays:
                padInput(grouped_arrays[key])
            processedInput = grouped_arrays
            if((label in X_train_by_label) == False):
                X_train_by_label[label] = {key:[] for key in processedInput}
            for key in processedInput:
                X_train_by_label[label][key] = X_train_by_label[label].get(key, []) + processedInput[key]
                
            y_train_by_label[label] = y_train_by_label.get(label, []) + ([label] * len(processedInput))
            
            #print(X_train_by_label[label])
        else:
            processedInput = padInput(grouped_arrays)
        #if( m > max_size):
        #    max_size =m
        
        print(len(X_train_by_label[label]))
    X_train = {key:[] for key in y_train_by_label[label]}
    y_train = []
    #print(X_train_by_label[0])
    #Truncate the data so that we have the same amount in each catagory
    minimumN = min([len(X_train_by_label[label][11.0]) for label in X_train_by_label])
    print("mininumM:", minimumN)
    for label in X_train_by_label:
        for key in X_train_by_label[label]:
            X_train[key] = X_train[key] + X_train_by_label[label][key][:minimumN]
        y_train = y_train + y_train_by_label[label][:minimumN]
    X_train_by_label = None
    y_train_by_label = None
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    indices = np.arange(len(y_train))
    np.random.shuffle(indices)
    X_train = X_train[indices]
    y_train = y_train[indices]
   
    print("MAX_SIZE:", max_size)
    return X_train, y_train



using gpu1
using theano


In [18]:
nFiles = 2
ttbar_files = glob.glob("/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_joined/*.h5")
WJet_files = glob.glob("/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_joined/*.h5")
qcd_files = glob.glob("/data/shared/Delphes/qcd_lepFilter_13TeV/pandas_joined/*.h5")
#print(WJet_files)
files = []
for i in range(nFiles):
    files.append((ttbar_files[i+1],0))
    files.append((WJet_files[i+1],1))

print(files)

[('/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_joined/ttbar_lepFilter_13TeV_1.h5', 0), ('/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_joined/wjets_lepFilter_13TeV_10.h5', 1), ('/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_joined/ttbar_lepFilter_13TeV_10.h5', 0), ('/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_joined/wjets_lepFilter_13TeV_100.h5', 1)]


In [19]:
X_train, y_train = preprocessFromPandas_file_label_pairs(files, cull=False)

6
5
6
6


KeyError: 11.0

In [ ]:
X_train


In [ ]:
histories = {}
cull=False
add_title = "_AllParticles"
for i in range (1):
   
    X_train, y_train = preprocessFromPandas_file_label_pairs(files, cull=cull)
    X_train_flatten = np.array([np.ndarray.flatten(x) for x in X_train])
    #DENSE
    dense = Sequential()
    dense.add(Dense(10, input_dim=set_size * vecsize,activation='relu'))
    #model.add(Dropout(.5))
    dense.add(Dense(1, activation='sigmoid'))
    dense.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    #CONVOLUTIONAL
    #conv = Sequential()
    #conv.add(Convolution2D(40,4,4, input_shape=(1,set_size,vecsize),activation='relu'))
    #conv.add(Flatten())
    #conv.add(Dense(1, activation='sigmoid'))
    #conv.compile(loss='binary_crossentropy',
    #              optimizer='rmsprop',
    #              metrics=['accuracy'])
    
    earlyStopping = EarlyStopping(verbose=1, patience=10)
    overfitStopping = OverfitStopping(verbose=1, patience=20)
    #smartCheckpoint = SmartCheckpoint("dense"+add_title)
    #RUN Dense
    dense_history = dense.fit(X_train_flatten, y_train,
                        batch_size=batch_size,
                        nb_epoch=epochs,
                        validation_split=.2,
                        callbacks=[earlyStopping, overfitStopping])
    histories["dense"+add_title] = (dense,dense_history,X_train_flatten, y_train)
#   plot_history([("dense",dense_history)])

    
    #earlyStopping = EarlyStopping(verbose=1, patience=10)
    #overfitStopping = OverfitStopping(verbose=1, patience=10)
    #Run Conv
    #conv_history = conv.fit(np.reshape(X_train, (len(X_train), 1, set_size, vecsize)), y_train,
    #                    batch_size=batch_size,
    #                    nb_epoch=epochs,
    #                    validation_split=.2,
    #                    callbacks=[earlyStopping, overfitStopping])
    #plot_history([("conv",conv_history)])
    #histories["conv"+add_title] = (conv,conv_history,X_train, y_train)
    #add_title = "_ObservableOnly"
    #cull=True

In [ ]:
keys = [key for key in histories]
def p(key):
    tup = histories[key]
    model = tup[0]
    history = tup[1]
    #print_accuracy_m(model, tup[2], tup[3])
    print(key + ': Best Validation accuracy: %r%%' % max(history.history["val_acc"]))
    plot_history([(key, history)])


In [ ]:
p(keys[0])

In [ ]:
p(keys[1])

In [ ]:
p(keys[2])

In [ ]:
p(keys[3])